In [81]:
! pip install psycopg2 

  Using cached psycopg2-2.9.6-cp310-cp310-win_amd64.whl (1.2 MB)


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\Users\\eduar\\Documents\\NoSQL_Lab_and_MongoDB\\nosql_lab\\venv\\Lib\\site-packages\\psycopg2\\_psycopg.cp310-win_amd64.pyd'
Check the permissions.



In [86]:
import datetime
import time
import logging
import json
import os
import psycopg2
import pandas as pd
import requests
import numpy as np

#from airflow import DAG
#from airflow.operators.python_operator import PythonOperator
#from airflow.operators.postgres_operator import PostgresOperator
#from airflow.hooks.postgres_hook import PostgresHook
#from airflow.timetables.trigger import CronTriggerTimetable

# Database credentials
ENDPOINT = 'sharedmobility.cnls2nxrzngg.us-east-1.rds.amazonaws.com'   #Nithu's shared point
DB_NAME = 'sharedmobility'
USERNAME = 'nithu'
PASSWORD = 'DWH_2023'


# Connect to the database
try:
    print("host={} dbname={} user={} password={}".format(ENDPOINT, DB_NAME, USERNAME, PASSWORD))
    conn = psycopg2.connect("host={} dbname={} user={} password={}".format(ENDPOINT, DB_NAME, USERNAME, PASSWORD))

except psycopg2.Error as e:
    print("Error: Could not make connection to the Postgres database")
    print(e)


# Get cursor to the database    
try:    
    cur = conn.cursor()

except psycopg2.Error as e:
    print("Error: Could not get curser to the Database")
    print(e)

# Auto commit to the database
conn.set_session(autocommit=True)

# Fetch table from database
try:
    data = pd.read_sql('SELECT * FROM statusdb', conn)    
    #cur.execute("SELECT * FROM statusdb")
except psycopg2.Error as e:
    print("Error: Could not get data from the database")
    print(e)

# Close the connection
cur.close()
conn.close()

data.head()

host=sharedmobility.cnls2nxrzngg.us-east-1.rds.amazonaws.com dbname=sharedmobility user=nithu password=DWH_2023


C:\Users\eduar\AppData\Local\Temp\ipykernel_23236\787720625.py:47: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql('SELECT * FROM statusdb', conn)


,load,station_id,is_installed,is_renting,is_returning,last_reported,num_bikes_available,num_docks_available,provider_id
0,2023-04-19 14:21:41.445570,emobility:69636,true,true,true,1681913867,0,0,emobility
1,2023-04-19 14:21:41.445570,emobility:69644,true,true,true,1681913867,3,0,emobility
2,2023-04-19 14:21:41.445570,emobility:69652,true,true,true,1681913867,1,0,emobility
3,2023-04-19 14:21:41.445570,emobility:69677,true,true,true,1681913867,2,0,emobility
4,2023-04-19 14:21:41.445570,emobility:69682,true,true,true,1681913867,1,0,emobility


In [94]:
set_station_id = set(data['station_id'])
set_provider_id = set(data['provider_id'])
print(f'We have {len(set_provider_id)} unique providers in our dataset' )
print(f'We have {len(set_station_id)} unique stations in our dataset' )
set_station_id

We have 30 unique providers in our dataset
We have 6100 unique stations in our dataset


{'mobility:70195',
 'publibike:452',
 'publibike:335',
 'donkey_ge:21341',
 'publiebike:813',
 '2em_cars:4812',
 'publibike:465',
 '2em_cars:3045',
 'mobility:82822',
 '2em_cars:4928',
 'publiebike:793',
 '2em_cars:6649',
 'mobility:71144',
 'publibike:449',
 'emobility:83128',
 'donkey_ge:21465',
 '2em_cars_e:4904',
 'publiebike:726',
 'publiebike:590',
 'publiebike:515',
 'liemobil_liechtenstein_ebike:13',
 'publiebike:415',
 'publibike:815',
 'mobility:70060',
 'publiebike:41',
 '2em_cars_e:4576',
 'mobility:70097',
 'sponticar:1132',
 'donkey_ge:21896',
 '2em_cars:5380',
 'publibike:674',
 'mobility:71594',
 '2em_cars:6733',
 'donkey_thun:20031',
 'publibike:638',
 'emobility:81299',
 'nextbike_ch:58622941',
 'publibike:379',
 'edrivecarsharing:1224',
 'donkey_ge:21968',
 '2em_cars:1575',
 '2em_cars:6254',
 'carvelo2go:beeb3043-a30e-43bd-a3ea-562587d2c344',
 'publiebike:424',
 'mobility:69706',
 'donkey_ge:22310',
 'carvelo2go:1f20dbbd-a186-4ae4-ae0b-0236e4770a0f',
 'publiebike:327

In [96]:
# create a new column called 'usage' with the difference between the current num_bikes_available and the previous one
data['usage'] = data.groupby(['station_id'])['num_bikes_available'].diff()*(-1)
columns_list = ['load','station_id', 'num_bikes_available', 'usage']
# Set negative values to 0
data['usage'] = data['usage'].apply(lambda x: max(0, x))
usage = data[columns_list]
# Check one single station_id
usage[usage_df['station_id'] == 'publibike:449']

,load,station_id,num_bikes_available,usage
2594,2023-04-19 14:27:50.527497,publibike:449,0,0.0
11194,2023-04-20 09:30:58.563099,publibike:449,0,0.0
17313,2023-04-20 11:04:59.311721,publibike:449,0,0.0
23413,2023-04-20 12:19:38.870544,publibike:449,0,0.0
29513,2023-04-20 13:34:23.248268,publibike:449,0,0.0
35318,2023-04-20 17:48:29.413665,publibike:449,0,0.0
40479,2023-04-20 17:54:54.138678,publibike:449,0,0.0
44965,2023-04-21 08:41:34.982879,publibike:449,2,0.0
51091,2023-04-21 09:18:58.331058,publibike:449,2,0.0
57121,2023-04-21 10:45:05.500322,publibike:449,1,1.0
